Import libraries

In [1]:
import pandas as pd
#!conda install beautifulsoup4
from bs4 import BeautifulSoup
import requests

Get table from Wikipedia page

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' # wiki page containing a table of Toronto neighborhoods
r = requests.get(url) 
soup = BeautifulSoup(r.content,"html5lib")  # html content of the url
tables = soup.find('table', class_ = 'wikitable sortable') # get table
rows = tables.find_all('tr') # get table rows

Loop over table to divide data to postcodes, boroughs and neighbourhoods

In [3]:
postcodes = []
boroughs = []
neighborhoods = []

for row in rows:
    col = row.find_all('td')
    try:
        if col[1].text != 'Not assigned': # skip rows with not assigned
            postcodes.append(col[0].text)
            boroughs.append(col[1].text)
            nbds_tmp = col[2].text.split('\n')[0] # remove new line tag
            if nbds_tmp == 'Not assigned': # if neighborhood isnt assigned, use borough instead
                neighborhoods.append(col[1].text)
            else:
                neighborhoods.append(nbds_tmp)    
    except: # first row
        pass
        


Create the dataframe

In [5]:
d = {'Postcode': postcodes, 'Borough': boroughs ,'Neighborhood': neighborhoods}
tor_nbds = pd.DataFrame(data=d)
tor_nbds.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


Aggregate rows with same postcode

In [6]:
tor_nbds = tor_nbds.groupby(['Postcode','Borough'])['Neighborhood'].agg([('Neighborhood', ', '.join)]).reset_index()
tor_nbds.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
print('Number of rows: ', tor_nbds.shape[0])
print('Number of cols: ', tor_nbds.shape[1])

Number of rows:  103
Number of cols:  3
